# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
 

from sklearn.metrics import classification_report, accuracy_score

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#cleaned data without non-attack values
X = pd.read_csv(root_path+'Data/Features.csv', index_col=0)
Y = pd.read_csv(root_path+'Data/Target.csv', index_col=0)

# Feature Engineering

In [0]:
# %%capture warnings1
# cluster and score
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.cluster import adjusted_rand_score

score = []
for i in range(len(X.columns)): # loop number of features
  K = MiniBatchKMeans(n_clusters=len(Y[Y.columns[0]].unique()), random_state=0)
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = adjusted_rand_score(Y[Y.columns[0]].values,pred)
  score.append(s)
  


In [5]:
# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 


print("Index of Feature , adjusted_rand_score\n")
for i in range(len(X.columns)):
    print(li[i])


Index of Feature , adjusted_rand_score

(3, 0.8593442140889673)
(9, 0.8159774538388821)
(18, 0.622075285370161)
(22, 0.5148510725654428)
(12, 0.5003701332075831)
(19, 0.4777702898635278)
(11, 0.4772264136222558)
(27, 0.47712220194294597)
(23, 0.4746163932672621)
(28, 0.4722968927431114)
(1, 0.47224526084922985)
(13, 0.4716586888127697)
(30, 0.4678951933262619)
(6, 0.46440849953320423)
(32, 0.4618912368596649)
(2, 0.4562089116717683)
(14, 0.4555514590826053)
(15, 0.45553461170762044)
(7, 0.44892169635195656)
(16, 0.4480196945385363)
(17, 0.44401959039295563)
(29, 0.43126537261064335)
(26, 0.3402657023313375)
(0, 0.2858463909077784)
(8, 0.257252250698114)
(10, 0.2516725467073946)
(5, 0.18686618395142254)
(4, 0.160936989667599)
(20, 0.1318495727125251)
(33, 0.13184655232101639)
(36, 0.10291737778729523)
(38, 0.10165025299509835)
(37, 0.08180156098873068)
(21, 0.06403794656341336)
(42, 0.054042832238337266)
(41, 0.04788436782471791)
(39, 0.04091933682365432)
(24, 0.01394425413614994)
(25, 

In [0]:
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]




In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

In [8]:
# RF w/out recursive Feature Elimination
x_train, x_test, y_train, y_test = train_test_split(X1, Y, random_state=11)
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred, digits=4))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                precision    recall  f1-score   support

      Analysis     0.6727    0.1793    0.2832       619
      Backdoor     0.6026    0.0829    0.1457       567
           DoS     0.3137    0.2663    0.2880      4075
      Exploits     0.6348    0.8064    0.7104     11236
       Fuzzers     0.8932    0.8667    0.8797      6127
       Generic     0.9968    0.9861    0.9914     53886
Reconnaissance     0.9213    0.7436    0.8230      3385
     Shellcode     0.6827    0.6211    0.6505       388
         Worms     0.8571    0.1579    0.2667        38

      accuracy                         0.8903     80321
     macro avg     0.7305    0.5234    0.5598     80321
  weighted avg     0.8935    0.8903    0.8874     80321



In [9]:
# %%capture warnings
# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(random_state=0, n_jobs=-1)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff

In [10]:
index = accuracy.index(max(accuracy))
# for i in accuracy:
#   print(i)
  

# print(accuracy)

print(max(accuracy), "max accuracy")

print(index, "index")


X3 = X2.iloc[:,0:len(X.columns)-index]

0.896882829886238 max accuracy
37 index


# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X3, Y, test_size=0.2, random_state=11)

In [12]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [13]:
x = list(zip(X3.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X3.columns)):
    print(x[i])

('sbytes', 0.27942957294482457)
('smeansz', 0.24990278174179906)
('service', 0.24256751575843097)
('Sjit', 0.09013841406002053)
('dmeansz', 0.07578455114574444)
('Spkts', 0.06217716434918047)


In [14]:
clf.score(x_train, y_train) 

0.9216110432407616

In [15]:
print("Accuracy")
clf.score(x_test, y_test) 

Accuracy


0.896882829886238

In [16]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

                precision    recall  f1-score   support

      Analysis     0.5859    0.1515    0.2408       495
      Backdoor     0.8333    0.0959    0.1721       469
           DoS     0.4978    0.1057    0.1744      3235
      Exploits     0.6060    0.9199    0.7307      8944
       Fuzzers     0.9056    0.8279    0.8650      4846
       Generic     0.9968    0.9881    0.9924     43219
Reconnaissance     0.9180    0.7430    0.8212      2696
     Shellcode     0.6128    0.6262    0.6194       321
         Worms     0.7037    0.5938    0.6441        32

      accuracy                         0.8969     64257
     macro avg     0.7400    0.5613    0.5845     64257
  weighted avg     0.9007    0.8969    0.8842     64257

